In [ ]:
#include libs

!pip install tensorflow

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
# baixando o data set

!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
!unzip cats_and_dogs_filtered.zip
!rm -rf cats_and_dogs_filtered.zip

In [ ]:
#definindo o dataset
IMAGE_SHAPE = (224, 224)
image_shape = (224, 224, 3)

##Define os diretórios para os dados de treinamento e validação
TRAINING_DATA_DIR = 'cats_and_dogs_filtered/train/'
VALID_DATA_DIR = 'cats_and_dogs_filtered/validation/'

class_names = ['gato', 'cachorro']

In [ ]:
#preparando o objeto para redimensionar os valores dos pixels das imagens entre 0 e 1

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [ ]:
#criando conjuntos de dados de treinamento e validação a partir dos diretorios

train_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE
)

valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE
)


In [ ]:
#plotando algumas imagens do conjunto de treinamento

def plot_dataset(dataset, num_images=5):
  batch = dataset.next()

  images = batch[0]
  labels = batch[1]

  random_indices = np.random.choice(len(images), num_images, replace=False)

  for i, idx in enumerate(random_indices):
    plt.subplot(1, num_images, i+1)
    plt.imshow(images[idx])
    plt.title(class_names[np.argmax(labels[idx])])
    plt.axis('off')
  plt.show()
plot_dataset(train_generator)

In [ ]:
# transfer learning

model_transfer_learning = tf.keras.applications.MobileNetV2(input_shape =
                                                            image_shape, include_top = False,
                                                            weights = 'imagenet')
model_transfer_learning.trainable = False
model_transfer_learning.summary()

In [ ]:
#montando o modelo

batch_size = 32
epochs = 10
learning_rate = 0.0001

## define o modelo da rede neural

model = tf.keras.Sequential([
    model_transfer_learning,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax'),
])

In [ ]:
#compule o modelo com otimizador, função de perda e metricas

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

print(model.summary())

In [ ]:
# treinamento

mc = ModelCheckpoint('model.h5', monitor='val-accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(train_generator, batch_size=batch_size, epochs=epochs, validation_data=valid_generator,
                    callbacks=[mc], verbose=1)

train_loss = history.history['loss']
train_acc = history.history['accuracy']
valid_loss = history.history['val_loss']
valid_acc = history.history['val_accuracy']